<a href="https://colab.research.google.com/github/vinayrvr987/RAGChat/blob/master/experiments/Agentic_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
!pip install langchain langchain_community wikipedia arxiv langchain_huggingface langchain_text_splitters chromadb langchain_groq

In [46]:
import os
from google.colab import userdata

GROQ_API_KEY = userdata.get('GROQ_API_KEY')
# Make sure your LangSmith API key is correctly set in Colab secrets.
LANGCHAIN_API_KEY = userdata.get('LANGCHAIN_API_KEY')

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_PROJECT"] = "Agentic AI exp"            # Optional but useful
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

# Add print statements to confirm environment variables are set
print(f"LANGCHAIN_TRACING_V2: {os.environ.get('LANGCHAIN_TRACING_V2')}")
print(f"LANGCHAIN_API_KEY is set: {os.environ.get('LANGCHAIN_API_KEY') is not None}")
print(f"LANGCHAIN_PROJECT: {os.environ.get('LANGCHAIN_PROJECT')}")
print(f"LANGCHAIN_ENDPOINT: {os.environ.get('LANGCHAIN_ENDPOINT')}")

LANGCHAIN_TRACING_V2: true
LANGCHAIN_API_KEY is set: True
LANGCHAIN_PROJECT: Agentic AI exp
LANGCHAIN_ENDPOINT: https://api.smith.langchain.com


### Creating Inbuild tools in Langchain

In [47]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

api_wrapper_wiki = WikipediaAPIWrapper()
api_wrapper_arxiv = ArxivAPIWrapper()

wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
arxiv_tool = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

tools = [wiki_tool, arxiv_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.11/dist-packages/wikipedia/__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000))]

### Custom Tools - RAG Tool

In [48]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
data = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(data)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma.from_documents(documents, embeddings)
retriever = db.as_retriever()
retriever


VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7c2c3096ead0>, search_kwargs={})

In [49]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever, "langchain-docs", "Searches the langchain docs for answers.")
retriever_tool

Tool(name='langchain-docs', description='Searches the langchain docs for answers.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7c2aa3e66ac0>, retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7c2c3096ead0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x7c2aa3e9a340>, retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7c2c3096ead0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_sep

In [50]:
tools=[wiki_tool, arxiv_tool, retriever_tool]
print(len(tools))
tools

3


[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.11/dist-packages/wikipedia/__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='langchain-docs', description='Searches the langchain docs for answers.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7c2aa3e66ac0>, retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7c2c3096ead0>, search_kwargs={}),

### Run the tools with Agents

In [51]:
from langchain_groq import ChatGroq
from google.colab import userdata

GROQ_API_KEY = userdata.get('GROQ_API_KEY')
llm = ChatGroq(groq_api_key=GROQ_API_KEY, model="llama-3.1-8b-instant")
llm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7c2ac1e4d510>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7c2ac1e4cd50>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [52]:
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [53]:
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)


In [54]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [55]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langchain-docs` with `{'query': 'Langsmith'}`


Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic ‚Äî¬†it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability‚Äã
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith‚Äôs observability features have yo

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

It appears that there is not enough information available about Langsmith in Wikipedia and Langchain-docs.

> Finished chain.


{'input': 'Tell me about Langsmith',
 'output': 'It appears that there is not enough information available about Langsmith in Wikipedia and Langchain-docs.'}

In [56]:
agent_executor.invoke({"input":"what is artificial intelligence?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Artificial intelligence'}`




Page: Artificial intelligence
Summary: Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.
High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general 

It seems like the text got cut off. I'll provide the full response.

Since the provided information does not fully answer your question, I'll provide more information on the topic.

Artificial general intelligence (AGI) is a type of artificial intelligence that aims to match or surpass human capabilities across virtually all cognitive tasks. This includes reasoning, problem-solving, learning, and decision-making, among others.

AGI is considered a more advanced form of artificial intelligence, as it is designed to be more versatile and able to perform a wide range of tasks, rather than being limited to a specific task or set of tasks.

There are different types of AGI, including:

1. Narrow or weak AGI: This type of AGI is designed to perform a specific task or set of tasks, but is not capable of general reasoning or problem-solving.
2. General or strong AGI: This type of AGI is designed to be capable of general reasoning and problem-solving, and is able to learn and adapt to new situa

{'input': 'what is artificial intelligence?',
 'output': "It seems like the text got cut off. I'll provide the full response.\n\nSince the provided information does not fully answer your question, I'll provide more information on the topic.\n\nArtificial general intelligence (AGI) is a type of artificial intelligence that aims to match or surpass human capabilities across virtually all cognitive tasks. This includes reasoning, problem-solving, learning, and decision-making, among others.\n\nAGI is considered a more advanced form of artificial intelligence, as it is designed to be more versatile and able to perform a wide range of tasks, rather than being limited to a specific task or set of tasks.\n\nThere are different types of AGI, including:\n\n1. Narrow or weak AGI: This type of AGI is designed to perform a specific task or set of tasks, but is not capable of general reasoning or problem-solving.\n2. General or strong AGI: This type of AGI is designed to be capable of general reaso